In [1]:
# 環境設置
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
import pandas as pd
import requests
import requests.packages.urllib3
requests.packages.urllib3.disable_warnings()

ISBN = 9789861206066  # 三體

df_lst = []  # 裝著各種表格的 list

my_options = Options()
my_options.add_argument("--incognito")  # 開啟無痕模式
# my_options.add_argument("--headless")  # 不開啟實體瀏覽器
driver = webdriver.Chrome(options=my_options)

In [2]:
# 搜尋
url = "http://140.131.94.8/uhtbin/webcat/"
driver.get(url)

select = Select(driver.find_element_by_name("srchfield1"))
select.select_by_visible_text(u"所有欄位")
search_input = driver.find_element_by_name("searchdata1")
search_input.send_keys(ISBN)
search_input.submit()

In [3]:
# 取得 dataframe
html_text = driver.page_source
dfs = pd.read_html(html_text, encoding="utf-8")
df_ntunhs = dfs[0]
df_ntunhs

,0,1,2,3,4
0,館藏,館藏,館藏,查看選項,查看選項
1,總館,總館,總館,總館,NaN
2,NaN,複本 館藏條碼,資料類型,館藏位置(現況),NaN
3,857.83 8729 v.1,1 1012962,中文圖書,二樓書庫,NaN
4,857.83 8729 v.2,1 1012963,中文圖書,二樓書庫,NaN
5,857.83 8729 v.3,1 1012964,中文圖書,二樓書庫,NaN


In [4]:
# 整理 dataframe
place = df_ntunhs.at[1, 0]
df_ntunhs.drop([0, 1, 2], inplace=True)
df_ntunhs.drop([1, 2, 4], axis=1, inplace=True)
df_ntunhs.insert(0, "圖書館", ["國立臺北護理健康大學" for i in range(df_ntunhs.shape[0])])
df_ntunhs.insert(1, "館藏地", [place for i in range(df_ntunhs.shape[0])])
df_ntunhs.rename(columns={0: "索書號",
                          3: "館藏狀態"}, inplace=True)
df_ntunhs.fillna(method="ffill", axis=0, inplace=True)  # 遇到值為 NaN時，將前一列的值填補進來
df_ntunhs.reset_index(inplace=True, drop=True)  # 重置索引值
df_ntunhs

,圖書館,館藏地,索書號,館藏狀態
0,國立臺北護理健康大學,總館,857.83 8729 v.1,二樓書庫
1,國立臺北護理健康大學,總館,857.83 8729 v.2,二樓書庫
2,國立臺北護理健康大學,總館,857.83 8729 v.3,二樓書庫
